# DB2-Salesforce connector: determine organization of each contact based on aggregated cluster and citation information.


In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'contact_citation_asso__c'

from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')

salesforce = application.new_salesforce_engine()
db_s = salesforce


nanoHUB - Serving Students, Researchers & Instructors


Obtained Salesforce access token ...... True


In [2]:
    
import pandas as pd
import time
import datetime

## Obtain tool information from DB2 

## Obtain Salesforce IDs


In [3]:
# get Salesforce ID for contact-citation associations
sf_citation_asso_df = db_s.query_data('SELECT Id, Organization__c, Contact__c, Citation__c FROM contact_citation_asso__c')

[Success] Bulk job creation successful. Job ID = 7505w00000b2CyLAAU


{"id":"7505w00000b2CyLAAU","operation":"query","object":"contact_citation_asso__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:18:27.000+0000","systemModstamp":"2021-10-25T11:18:27.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000b2CyLAAU","operation":"query","object":"contact_citation_asso__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:18:27.000+0000","systemModstamp":"2021-10-25T11:18:32.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":4944,"retries":0,"totalProcessingTime":536}
[Success] Bulk job completed successfully.


In [4]:
# get Salesforce ID for contact-cluster associations
sf_cluster_asso_df = db_s.query_data('SELECT Id, Contact__c, Tool_Usage_Cluster__c FROM ContactToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000b2D5QAAU


{"id":"7505w00000b2D5QAAU","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:18:42.000+0000","systemModstamp":"2021-10-25T11:18:42.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}


{"id":"7505w00000b2D5QAAU","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:18:42.000+0000","systemModstamp":"2021-10-25T11:18:47.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":98069,"retries":0,"totalProcessingTime":4672}
[Success] Bulk job completed successfully.


In [5]:
# get Salesforce ID for contacts
sf_contact_df = db_s.query_data('SELECT Id, Organization_Overwrite__c, Organization_email_derived__c FROM Contact')

[Success] Bulk job creation successful. Job ID = 7505w00000b2D1eAAE


{"id":"7505w00000b2D1eAAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:18:55.000+0000","systemModstamp":"2021-10-25T11:18:55.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000b2D1eAAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:18:55.000+0000","systemModstamp":"2021-10-25T11:18:56.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":14757,"retries":0,"totalProcessingTime":828}


{"id":"7505w00000b2D1eAAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:18:55.000+0000","systemModstamp":"2021-10-25T11:19:07.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":262499,"retries":0,"totalProcessingTime":11654}
[Success] Bulk job completed successfully.


In [6]:
# get Salesforce ID for citations
sf_citation_df = db_s.query_data('SELECT Id, Year__c FROM nanoHUB_citations__c')

[Success] Bulk job creation successful. Job ID = 7505w00000b2D5VAAU


{"id":"7505w00000b2D5VAAU","operation":"query","object":"nanoHUB_citations__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:19:22.000+0000","systemModstamp":"2021-10-25T11:19:22.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000b2D5VAAU","operation":"query","object":"nanoHUB_citations__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:19:22.000+0000","systemModstamp":"2021-10-25T11:19:23.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":2517,"retries":0,"totalProcessingTime":495}
[Success] Bulk job completed successfully.


In [7]:
# get Salesforce ID for clusters
sf_cluster_df = db_s.query_data('SELECT Id, Starting_Date__c FROM tool_usage_cluster__c')

[Success] Bulk job creation successful. Job ID = 7505w00000b2D5aAAE


{"id":"7505w00000b2D5aAAE","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:19:33.000+0000","systemModstamp":"2021-10-25T11:19:33.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000b2D5aAAE","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:19:33.000+0000","systemModstamp":"2021-10-25T11:19:36.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":4290,"retries":0,"totalProcessingTime":436}
[Success] Bulk job completed successfully.


## Determine domain by citation by latest citation

In [8]:
# display data used in this section
display(sf_citation_df.head(2))
display(sf_citation_asso_df.head(2))

,Id,Year__c
0,a0t5w000008p7pSAAQ,2012.0
1,a0t5w000008p7pTAAQ,2013.0


,Citation__c,Contact__c,Id,Organization__c
0,a0t5w000008p7pSAAQ,0035w000034JLpOAAW,a0u5w00000RhVhTAAV,a0r5w00000V42cCAAR
1,a0t5w000008p7q1AAA,0035w000034JLpOAAW,a0u5w00000RhVhUAAV,a0r5w00000V42cCAAR


In [9]:
overall_df = pd.merge(sf_citation_df, sf_citation_asso_df, \
                      how='inner', left_on='Id', right_on='Citation__c', suffixes=['', '_citation_asso'])

# display
overall_df.head(2)

,Id,Year__c,Citation__c,Contact__c,Id_citation_asso,Organization__c
0,a0t5w000008p7pSAAQ,2012.0,a0t5w000008p7pSAAQ,0035w000034JLpOAAW,a0u5w00000RhVhTAAV,a0r5w00000V42cCAAR
1,a0t5w000008p7pSAAQ,2012.0,a0t5w000008p7pSAAQ,0035w000034JLfwAAG,a0u5w00000RhVi2AAF,a0r5w00000V42cCAAR


In [10]:
# infer contact's organziation by using latest citation organization
org_from_citation_df = overall_df.groupby('Contact__c').apply(lambda x: \
                                                   x.sort_values(by=['Year__c'], ascending=False).head(1).Organization__c.iloc[0])\
                                            .dropna().reset_index(name='Organization_citation_derived__c')

# display
org_from_citation_df.head(2)

,Contact__c,Organization_citation_derived__c
0,0035w000031Vt92AAC,a0r5w00000V42pJAAR
1,0035w000031Vt9kAAC,a0r5w00000V42nyAAB


In [11]:
# Send data to Salesforce

df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Id']         = org_from_citation_df['Contact__c']
df_sf['Organization_citation_derived__c'] = org_from_citation_df['Organization_citation_derived__c']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Id,0035w000031Vt92AAC,0035w000031Vt9kAAC
Organization_citation_derived__c,a0r5w00000V42pJAAR,a0r5w00000V42nyAAB


In [12]:
db_s = salesforce

# specify Salesforce object ID and external ID
db_s.object_id = 'Contact'
db_s.external_id = 'Id'

In [13]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b2CznAAE
hello


[Success] CSV upload successful. Job ID = 7505w00000b2CznAAE


[Success] Closing job successful. Job ID = 7505w00000b2CznAAE


In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-25T11:19:47.000+0000',
 'externalIdFieldName': 'Id',
 'id': '7505w00000b2CznAAE',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'Contact',
 'operation': 'upsert',
 'retries': 0,
 'state': 'InProgress',
 'systemModstamp': '2021-10-25T11:19:48.000+0000',
 'totalProcessingTime': 0}


## Determine cluster's domain by contact information

In [15]:
# display data used in this section
display(sf_cluster_asso_df.head(2))
display(sf_contact_df.head(2))

,Contact__c,Id,Tool_Usage_Cluster__c
0,0035w000034JG1EAAW,a0x5w00000cMzV5AAK,a0w5w000009Q7m3AAC
1,0035w000034I5sjAAC,a0x5w00000cMzVAAA0,a0w5w000009Q7m3AAC


,Id,Organization_Overwrite__c,Organization_email_derived__c
0,0035w000031Vsp1AAC,NaN,NaN
1,0035w000031Vsp2AAC,NaN,a0r5w00000V42cCAAR


In [16]:
# Convert time to datetime in Pandas
sf_cluster_df['Starting_Date__c'] = pd.to_datetime(sf_cluster_df.Starting_Date__c)

In [17]:
overall_df = pd.merge(sf_cluster_asso_df[['Contact__c', 'Tool_Usage_Cluster__c']], sf_contact_df, \
                                     how ='inner', left_on='Contact__c', right_on='Id' )

overall_df.drop(columns=['Id', 'Contact__c'], inplace=True)

# display
overall_df.head(2)

,Tool_Usage_Cluster__c,Organization_Overwrite__c,Organization_email_derived__c
0,a0w5w000009Q7m3AAC,NaN,a0r5w00000V42cCAAR
1,a0w5w000009Q7lsAAC,NaN,a0r5w00000V42cCAAR


In [18]:
# apply overwrite
overall_df['final_org'] = overall_df.fillna('').apply(lambda x: \
                                    x.Organization_Overwrite__c if x.Organization_Overwrite__c!='' \
                                    else x.Organization_email_derived__c, axis=1)

# display
overall_df.head(2)

,Tool_Usage_Cluster__c,Organization_Overwrite__c,Organization_email_derived__c,final_org
0,a0w5w000009Q7m3AAC,NaN,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR
1,a0w5w000009Q7lsAAC,NaN,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR


In [19]:
# infer the organization of this cluster through majority vote
org_from_cluster_df = overall_df.groupby('Tool_Usage_Cluster__c').final_org.apply(lambda x: \
                                                   x.mode().iloc[0])\
                                            .dropna().reset_index(name='Organization__c')

# display
org_from_cluster_df.head(2)

,Tool_Usage_Cluster__c,Organization__c
0,a0w5w000009Q7CuAAK,
1,a0w5w000009Q7CvAAK,


In [20]:
# Send data to Salesforce

df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Id']         = org_from_cluster_df['Tool_Usage_Cluster__c']
df_sf['Organization__c'] = org_from_cluster_df['Organization__c']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Id,a0w5w000009Q7CuAAK,a0w5w000009Q7CvAAK
Organization__c,,


In [21]:
# create DB2 to Salesforce API object
db_s = salesforce

# specify Salesforce object ID and external ID
db_s.object_id = 'tool_usage_cluster__c'
db_s.external_id = 'Id'

In [22]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b2CzeAAE
hello


[Success] CSV upload successful. Job ID = 7505w00000b2CzeAAE


[Success] Closing job successful. Job ID = 7505w00000b2CzeAAE


In [23]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-25T11:19:52.000+0000',
 'externalIdFieldName': 'Id',
 'id': '7505w00000b2CzeAAE',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'tool_usage_cluster__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'InProgress',
 'systemModstamp': '2021-10-25T11:19:53.000+0000',
 'totalProcessingTime': 0}


## Determine domain by cluster by latest cluster

In [24]:
# display data used in this section
display(sf_cluster_df.head(2))
display(sf_cluster_asso_df.head(2))
display(org_from_cluster_df.head(2))
display(sf_contact_df.head(2))

,Id,Starting_Date__c
0,a0w5w000009Q7CuAAK,2010-09-05
1,a0w5w000009Q7CvAAK,2018-11-29


,Contact__c,Id,Tool_Usage_Cluster__c
0,0035w000034JG1EAAW,a0x5w00000cMzV5AAK,a0w5w000009Q7m3AAC
1,0035w000034I5sjAAC,a0x5w00000cMzVAAA0,a0w5w000009Q7m3AAC


,Tool_Usage_Cluster__c,Organization__c
0,a0w5w000009Q7CuAAK,
1,a0w5w000009Q7CvAAK,


,Id,Organization_Overwrite__c,Organization_email_derived__c
0,0035w000031Vsp1AAC,NaN,NaN
1,0035w000031Vsp2AAC,NaN,a0r5w00000V42cCAAR


In [25]:
overall_df = pd.merge(sf_cluster_asso_df, org_from_cluster_df,\
                      how='inner', left_on='Tool_Usage_Cluster__c', right_on='Tool_Usage_Cluster__c')
overall_df.drop(columns='Id', inplace=True)

# join cluster date
overall_df = pd.merge(overall_df, sf_cluster_df, \
                     how='inner', left_on='Tool_Usage_Cluster__c', right_on='Id')
overall_df.drop(columns='Id', inplace=True)

# display
overall_df.head(2)

,Contact__c,Tool_Usage_Cluster__c,Organization__c,Starting_Date__c
0,0035w000034JG1EAAW,a0w5w000009Q7m3AAC,a0r5w00000V42cCAAR,2019-01-05
1,0035w000034I5sjAAC,a0w5w000009Q7m3AAC,a0r5w00000V42cCAAR,2019-01-05


In [26]:
# infer the organization of this cluster through majority vote
contact_org_from_cluster_df = overall_df.groupby('Contact__c').apply(lambda x: \
                                                   x.sort_values(by=['Starting_Date__c'], ascending=False).head(1).Organization__c.iloc[0])\
                                            .dropna().reset_index(name='Organization_cluster_derived__c')

# display
contact_org_from_cluster_df.head(2)

,Contact__c,Organization_cluster_derived__c
0,0035w000031Vsp8AAC,a0r5w00000V42cCAAR
1,0035w000031VspAAAS,a0r5w00000V42cCAAR


In [27]:
contact_org_from_cluster_df = contact_org_from_cluster_df[contact_org_from_cluster_df.Organization_cluster_derived__c!='']

In [28]:
# Send data to Salesforce

df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Id']         = contact_org_from_cluster_df['Contact__c']
df_sf['Organization_cluster_derived__c'] = contact_org_from_cluster_df['Organization_cluster_derived__c']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Id,0035w000031Vsp8AAC,0035w000031VspAAAS
Organization_cluster_derived__c,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR


In [29]:
db_s = salesforce

# specify Salesforce object ID and external ID
db_s.object_id = 'Contact'
db_s.external_id = 'Id'

In [30]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b2D4TAAU
hello


[Success] CSV upload successful. Job ID = 7505w00000b2D4TAAU


[Success] Closing job successful. Job ID = 7505w00000b2D4TAAU


In [31]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-25T11:20:18.000+0000',
 'externalIdFieldName': 'Id',
 'id': '7505w00000b2D4TAAU',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'Contact',
 'operation': 'upsert',
 'retries': 0,
 'state': 'InProgress',
 'systemModstamp': '2021-10-25T11:20:20.000+0000',
 'totalProcessingTime': 0}


## Determine domain by cluster by latest cluster